In [1]:
import pandas as pd

import time
import torchvision
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm

from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler
import os

device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
from PIL import Image, ImageFile
from torch.utils.data import Dataset
from torchvision import transforms

import matplotlib.pyplot as plt


In [15]:
import numpy as np

In [3]:
device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [4]:
class RetinopathyDatasetTrain(Dataset):
    
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
        
        
    def __getitem__(self, idx):
        label = torch.tensor(self.data.loc[idx, "diagnosis"])
        img_name = os.path.join('/kaggle/input/train_images', 
                                self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
       # image = image.resize((256,256), resample=Image.BILINEAR)
        
        if self.transform:
            image = self.transform(image)
        
        sample = {'image': image, "label": label}
            
        return sample
        
        

In [5]:
class RetinopathyDatasetTest(Dataset):
    
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
        
        
    def __getitem__(self, idx):
        img_name = os.path.join('/kaggle/input/test_images', 
                                self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
       # image = image.resize((256,256), resample=Image.BILINEAR)
        
        if self.transform:
            image = self.transform(image)
            
        return {'image': image}

In [ ]:
def show_photo(image, label):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.pause(0.001)  # pause a bit so that plots are updated
    print(label)

In [6]:
transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


In [ ]:
dataset[0]["image"].shape

In [7]:
print(device)

cuda:0


In [18]:
model = torchvision.models.resnet152(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(2048, 1)

model = model.to(device)

In [9]:
dataset = RetinopathyDatasetTrain(csv_file = "/kaggle/input/train.csv", transform = transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 16, shuffle = True, num_workers = 4)

plist = [
         {'params': model.layer4.parameters(), 'lr': 1e-4, 'weight': 0.001},
         {'params': model.fc.parameters(), 'lr': 1e-3}
         ]

optimizer = optim.Adam(plist, lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10)

In [12]:
since = time.time()
criterion = nn.MSELoss()
num_epochs = 15
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    scheduler.step()
    model.train()
    running_loss = 0.0
    tk0 = tqdm(dataloader, total=int(len(dataloader)))
    counter = 0
    for bi, d in enumerate(tk0):
        inputs = d["image"]
        labels = d["label"].view(-1, 1)
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        counter += 1
        tk0.set_postfix(loss=(running_loss / (counter * dataloader.batch_size)))
    epoch_loss = running_loss / len(dataloader)
    print('Training Loss: {:.4f}'.format(epoch_loss))

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
torch.save(model.state_dict(), "model.bin")
        

  0%|          | 0/229 [00:00<?, ?it/s]

Epoch 0/14
----------


  0%|          | 0/229 [00:00<?, ?it/s]

Training Loss: 21.9540
Epoch 1/14
----------


  0%|          | 0/229 [00:00<?, ?it/s]

...

Training Loss: 15.7971
Epoch 13/14
----------


  0%|          | 0/229 [00:00<?, ?it/s]

Training Loss: 15.9358
Epoch 14/14
----------


100%|██████████| 229/229 [04:55<00:00,  1.09it/s, loss=0.964]


Training Loss: 15.4257
Training complete in 73m 56s


In [13]:
test_dataset = RetinopathyDatasetTest(csv_file = "/kaggle/input/test.csv", transform = transform)


In [20]:
%%time
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

# Prediction
with torch.no_grad():
    test_preds1 = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds1[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

CPU times: user 4.28 s, sys: 768 ms, total: 5.05 s
Wall time: 53.2 s


In [21]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

# Prediction
with torch.no_grad():
    test_preds2 = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds2[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [22]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

# Prediction
with torch.no_grad():
    test_preds3 = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds3[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [23]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

# Prediction
with torch.no_grad():
    test_preds4 = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds4[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [24]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

# Prediction
with torch.no_grad():
    test_preds5 = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds5[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [25]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

# Prediction
with torch.no_grad():
    test_preds6 = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds6[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [26]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

# Prediction
with torch.no_grad():
    test_preds7 = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds7[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [27]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

# Prediction
with torch.no_grad():
    test_preds8 = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds8[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [28]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

# Prediction
with torch.no_grad():
    test_preds9 = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds9[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [29]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 16, shuffle = True, num_workers = 4)

# Prediction
with torch.no_grad():
    test_preds10 = np.zeros((len(test_dataset), 1))
    
    for i, image in enumerate(test_dataloader):
        image = image["image"]
        pred = model(image.to(device))
        test_preds10[i * 16:(i + 1) * 16] = pred.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)

In [30]:
test_preds = (test_preds1 + test_preds2 + test_preds3 + test_preds4 + test_preds5
             + test_preds6 + test_preds7 + test_preds8 + test_preds9 + test_preds10) / 10.0

In [31]:

coef = [0.5, 1.5, 2.5, 3.5]

for i, pred in enumerate(test_preds):
    if pred < coef[0]:
        test_preds[i] = 0
    elif pred >= coef[0] and pred < coef[1]:
        test_preds[i] = 1
    elif pred >= coef[1] and pred < coef[2]:
        test_preds[i] = 2
    elif pred >= coef[2] and pred < coef[3]:
        test_preds[i] = 3
    else:
        test_preds[i] = 4



In [33]:

sample = pd.read_csv("../input/sample_submission.csv")
sample.diagnosis = test_preds.astype(int)
sample.to_csv("submission.csv", index=False)